### Lab 05: Deutsch–Jozsa Algorithm – Constant or Balanced?

Construct a Deutsch–Jozsa quantum circuit to determine whether the Boolean function

![Formula](https://latex.codecogs.com/png.image?\dpi{110}&space;f(x_0,x_1,x_2,x_3)=(x_0\land&space;x_1)\oplus(x_2\oplus&space;x_3))

is constant or balanced, using a single quantum query. The function should be implemented as a quantum oracle within the circuit.

---

#### Problem Overview

- A function ![formula](https://latex.codecogs.com/png.image?\dpi{110}&space;f%3A%20%5C%7B0%2C1%5C%7D%5En%20%5Cto%20%5C%7B0%2C1%5C%7D) is:
  - **constant** if it returns the same output (always 0 or always 1) for all inputs.
  - **balanced** if it returns 0 for exactly half of the inputs, and 1 for the other half.

---

#### Expected Output

After executing the circuit:
- If the measurement result is the **all‑zero bitstring**, the function is **constant**.
- If any other result appears (i.e., **any non-zero bitstring**), the function is **balanced**.

This task illustrates the quantum advantage of the Deutsch–Jozsa algorithm by resolving constancy versus balance in a single oracle evaluation.


In [ ]:
# enable inline display of matplotlib plots in Jupyter notebooks
%matplotlib inline

from qiskit import QuantumCircuit, QuantumRegister
from qiskit_aer import AerSimulator
from qiskit.visualization import plot_histogram,plot_distribution
from qiskit.visualization import circuit_drawer

def deutsch_jozsa_oracle():
    """
    Oracle for the Boolean function:
        f(x0, x1, x2, x3) = (x0 AND x1) XOR (x2 XOR x3)

    The oracle implements the transformation:
        |x⟩|y⟩ → |x⟩|y ⊕ f(x)⟩

    In Boolean algebra, XOR is associative and commutative, so:
        (x0 AND x1) XOR (x2 XOR x3)
    can be rewritten as:
        ((x0 AND x1) XOR x2) XOR x3
    This form matches the implementation order in the code, where the target
    qubit `f` is first updated with (x0 AND x1), then XORed with x2, and finally
    XORed with x3.

    Barriers are included for visual clarity in circuit diagrams, separating
    the oracle from other steps in the Deutsch–Jozsa algorithm.
    """
    x = QuantumRegister(4, name='x')
    f = QuantumRegister(1, name='f')
    qc = QuantumCircuit(x, f, name="Oracle")

    qc.barrier()  # visual separation from previous circuit steps

    # (x0 AND x1) → XOR into target qubit f
    qc.ccx(x[0], x[1], f[0])

    # XOR x2 into target qubit
    qc.cx(x[2], f[0])

    # XOR x3 into target qubit
    qc.cx(x[3], f[0])

    qc.barrier()  # visual separation from subsequent circuit steps

    return qc

def deutsch_jozsa_circuit():
    """
    Builds the Deutsch–Jozsa circuit for 4 input qubits and 1 target qubit.

    The circuit applies Hadamard gates to all input qubits, prepares the target
    qubit in the |−⟩ state, inserts the oracle between two layers of Hadamards
    on the input qubits, and measures the result to determine whether the 
    function is constant or balanced.

    Returns:
        QuantumCircuit: The complete 5-qubit Deutsch–Jozsa circuit with measurements.
    """
    qc = QuantumCircuit(5, 4)

    # Step 1: Initialize input qubits to superposition
    for i in range(4):
        qc.h(i)

    # Step 2: Prepare target qubit in |−⟩ state
    qc.x(4)
    qc.h(4)

    # Step 3: Apply oracle
    oracle = deutsch_jozsa_oracle()
    qc.compose(oracle, inplace=True)

    # Step 4: Apply Hadamard to input qubits again
    for i in range(4):
        qc.h(i)

    # Step 5: Measure input qubits
    qc.measure(range(4), range(4))

    return qc

# ------------------------------------------------
#                main program
# ------------------------------------------------
# -- build and run circuit --
qc = deutsch_jozsa_circuit()
simulator = AerSimulator()
result = simulator.run(qc, shots=1000).result()
counts = result.get_counts(qc)

# -- display --
print("Measurement result:", counts)
circuit_drawer(qc, output="mpl")
plot_histogram(counts)